In [ ]:
from datetime import datetime
import requests
import json
from gql import gql, Client
import pandas as pd
import time
import numpy as np
from sys import argv

from gql.transport.requests import RequestsHTTPTransport

In [ ]:
## Query string used to get latest parcel transaction information
mystring = """
{{
  orders (first: 1000 orderBy: updatedAt, orderDirection: asc where: {{ status:sold category:parcel updatedAt_gt:"{0}"}}, subgraphError: allow) {{
 
 
    category
    status
    price
    id
    updatedAt
    nft {{
      owner {{
        id
      }}
      name
      estate {{
        id
      }}
      parcel {{
        id
        x
        y
      }}
     
      owner {{
        id
      }}
    }}
   
  }}
 
}}"""

In [ ]:
df = pd.DataFrame()

#update parameter used in mystring to start querying the database at the earliest update date of sale. The update 
#date is specified in epoch date and needs to be converted to datetime for human consumption.
update = 1

while True:
    
    #query the data using GraphQL python library.
    query = gql(mystring.format(update))
    result = client.execute(query)
    
    #if there is no data returned it means you reached the end and should stop querying.
    if len(client.execute(query)['orders']) <= 1:
        break
   
    else:
        #Create a temporary dataframe to later append to the final dataframe that compiles all 1000-row dataframes.
        df1 = pd.DataFrame()
        df1 = pd.DataFrame(result['orders'])
        #unfold a subdict into a series of columns.
        df1 = df1.join(df1['nft'].apply(pd.Series),lsuffix='_1',rsuffix='_2')     
        
        #append your temp dataframe to your master dataset.
        df = df.append(df1)
        
        #Pass into the API the max date from your dataset to fetch the next 1000 records.
        update = df['updatedAt'].max()
        print("last updated at: {}".format(time.strftime('%Y-%m-%d', time.localtime(int(update)))))

#reformat the update date in human-readable datetime format.
df['updatedAt_dt'] = df['updatedAt'].apply(lambda x: time.strftime('%Y-%m-%d', time.localtime(int(x))) )
df['price'] = df['price'].astype(float)/1e18
df['owner'] = df.owner.apply(lambda x: x['id'])
df['x'] = df.parcel.apply(lambda a: a['x'])
df['y'] = df.parcel.apply(lambda a: a['y'])
df['parcel_id'] = df.parcel.apply(lambda a: a['id'])
df.to_csv('parcel_transactions.csv')

In [ ]:
# String used to query parcel info
mystring = """
{{
  parcels (first: 301 orderBy: x where: {{ y_lte:"{0}" y_gte:"{0}"}} , subgraphError: allow) {{
    id
    tokenId
    owner{{
     id
    }}
    x
    y
    estate{{
     id
    }}
  }}
 
}}"""

In [ ]:
df = pd.DataFrame()

#update parameter used in mystring to start querying the database at the earliest update date of sale. The update 
#date is specified in epoch date and needs to be converted to datetime for human consumption.
counter = 1

for row_num in range(-150,151):
    
    #query the data using GraphQL python library.
    query = gql(mystring.format(row_num))
    result = client.execute(query)
    
    #if there is no data returned it means you reached the end and should stop querying.
    if len(client.execute(query)['parcels']) < 1:
        break
   
    else:
        #Create a temporary dataframe to later append to the final dataframe that compiles all 1000-row dataframes.
        df1 = pd.DataFrame()
        df1 = pd.DataFrame(result['parcels'])
        
        """
        #unfold a subdict into a series of columns.
        df1 = df1.join(df1['nft'].apply(pd.Series),lsuffix='_1',rsuffix='_2')     
        """
        #append your temp dataframe to your master dataset.
        df = df.append(df1)
        
        #Pass into the API the max date from your dataset to fetch the next 1000 records.
        print('first {} rows done!'.format(row_num + 151))
df['owner'] = df.owner.apply(lambda x: x['id'])
df['x'] = df.x.apply(lambda a: int(a))
df['y'] = df.y.apply(lambda a: int(a))
df.to_csv('decentraland_ownership.csv')

In [ ]:
# String used to query estate info
mystring = """{{
  orders (first: 1000 orderBy: updatedAt, orderDirection: asc where: {{ status:sold category:estate updatedAt_gt:"{0}"}}, subgraphError: allow) {{
    category
    price
    id
    buyer
    updatedAt
    nft{{
      estate{{
        id
        parcels{{
        id
        x
        y
        }}
      }}
    }}
  }}
}}"""

In [ ]:
df = pd.DataFrame()

#update parameter used in mystring to start querying the database at the earliest update date of sale. The update 
#date is specified in epoch date and needs to be converted to datetime for human consumption.
update = 1

while True:
    
    #query the data using GraphQL python library.
    query = gql(mystring.format(update))
    result = client.execute(query)
    
    #if there is no data returned it means you reached the end and should stop querying.
    if len(client.execute(query)['orders']) <= 1:
        break
   
    else:
        #Create a temporary dataframe to later append to the final dataframe that compiles all 1000-row dataframes.
        df1 = pd.DataFrame()
        df1 = pd.DataFrame(result['orders'])
        #unfold a subdict into a series of columns.
        #df1 = df1.join(df1['nft'].apply(pd.Series),lsuffix='_1',rsuffix='_2')     
        
        #append your temp dataframe to your master dataset.
        df = df.append(df1)
        
        #Pass into the API the max date from your dataset to fetch the next 1000 records.
        update = df['updatedAt'].max()
        print("last updated at: {}".format(time.strftime('%Y-%m-%d', time.localtime(int(update)))))

#reformat the update date in human-readable datetime format.
df['updatedAt_dt'] = df['updatedAt'].apply(lambda x: time.strftime('%Y-%m-%d', time.localtime(int(x))) )
df['price'] = df['price'].astype(float)/1e18
df['parcels'] = df.nft.apply(lambda x: x['estate']).apply(lambda x: x['parcels']).apply(lambda x: [(int(i['x']),int(i['y'])) for i in x])
df['estate_id'] = df.nft.apply(lambda x: x['estate']).apply(lambda x: x['id'])
df.to_csv('estate_transactions.csv',index=False)